In [9]:
import cv2
import numpy as np

### 방법 1 or 2 中 택 1 (하나 하고 restart)
- 방법 1
1. 이미지 읽어오면 왼 마우스클릭으로 영역 지정
2. 엔터키 누르면 배경 제거된 개체(dst 창) 생성
3. 창 닫으면 저장

- 방법 2
1. 이미지 읽어오면 왼 마우스클릭으로 영역 지정
2. 엔터키 누르면 배경 제거된 개체(dst 창) 생성
3. 왼클릭: 파란색 펜, 객체 살리기 | 우클릭: 빨간색 펜, 배경 지우기
4. 연속 클릭&그리기 가능, 다했으면 엔터키 눌러 갱신
4. ESC키 누르면 창 닫히며 저장

In [ ]:
# 1. 배경제거 영역만 지정해서 masking
# 입력 영상 불러오기
src = cv2.imread('oneeyed.jpg')
src = cv2.resize(src, (900, 600))

if src is None:
    print('Image load failed!')
    sys.exit()
    
# 사장형 지정을 통한 초기 분할
rc = cv2.selectROI(src) # 초기 위치 지정하고 모서리 좌표 4개를 튜플값으로 반환
mask = np.zeros(src.shape[:2], np.uint8) # 마스크는 검정색으로 채워져있고 입력 영상과 동일한 크기

# 결과를 계속 업데이트 하고 싶으면 bgd, fgd 입력
cv2.grabCut(src, mask, rc, None, None, 5, cv2.GC_INIT_WITH_RECT)

# grabCut 자료에서 0,2는 배경, 1,3은 전경
# mask == 0 or mask == 2를 만족하면 0으로 설정 or 1로 설정
mask2 = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')

# np.newaxis로 차원 확장
dst = src * mask2[:, :, np.newaxis]

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite('removedBG.jpg', dst)

In [12]:
# 2. ppt에서 배경제거하듯이 배경제거 영역 지정 후 마우스로 세부 영역 지정
# 입력 영상 불러오기
src2 = cv2.imread('oneeyed.jpg')
src = cv2.resize(src2, (900, 600)) # 필요할 시 resize

if src is None:
    print('Image load failed!')
    sys.exit()
    
# 사각형 지정을 통한 초기 분할
mask = np.zeros(src.shape[:2], np.uint8) # 마스크
bgdModel = np.zeros((1, 65), np.float64) # 배경 모델 무조건 1행 65열, float64
fgdModel = np.zeros((1, 65), np.float64) # 전경 모델 무조건 1행 65열, float64

rc = cv2.selectROI(src)

# RECT는 사용자가 사각형 지정. 이 값에서 계속 업데이트
cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)

# mask 4개 값을 2개로 변환
mask2 = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')
dst = src * mask2[:, :, np.newaxis]

# 초기 분할 결과 출력
cv2.imshow('dst', dst)

# 마우스 이벤트 처리 함수 등록
def on_mouse(event, x, y, flags, param):
    
    if event == cv2.EVENT_LBUTTONDOWN: # 왼쪽 버튼은 전경
        cv2.circle(dst, (x, y), 3, (255, 0, 0), -1) # 파랑색 색칠
        cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1) # 마스크에 전경 강제 지정
        cv2.imshow('dst', dst)
    elif event == cv2.EVENT_RBUTTONDOWN: # 오른쪽 버튼은 배경
        cv2.circle(dst, (x, y), 3, (0, 0, 255), -1) # 빨강색 색칠
        cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1) # 마스크에 배경 강제 지정
        cv2.imshow('dst', dst)
        
    elif event == cv2.EVENT_MOUSEMOVE: # 마우스 움직임
        if flags & cv2.EVENT_FLAG_LBUTTON: # 왼쪽 누르고 움직이면 전경
            cv2.circle(dst, (x, y), 3, (255, 0, 0), -1)
            cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)
            cv2.imshow('dst', dst)
        elif flags & cv2.EVENT_FLAG_RBUTTON: # 오른쪽 누르고 움직이면 배경
            cv2.circle(dst, (x, y), 3, (0, 0, 255), -1)
            cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)
            cv2.imshow('dst', dst)

cv2.setMouseCallback('dst', on_mouse)

while True:
    key = cv2.waitKey()
    if key == 13:    #CR문자 '\r'
        cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_MASK) # 마스크 초기화
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
        dst = src * mask2[:, :, np.newaxis]
        cv2.imshow('dst', dst)

    elif key == 27: #ESC키
        break

cv2.destroyAllWindows()
cv2.imwrite('finalremovedBG.jpg', dst)

True

출처: https://deep-learning-study.tistory.com/240?category=946336 <br>
      https://github.com/sunkyoo/opencv4cvml <br>
      https://docs.opencv.org/4.3.0/d8/d83/tutorial_py_grabcut.html